<a href="https://colab.research.google.com/github/Lee-ane/Search-and-Filter-DataFrames/blob/main/Search_and_Filter_DataFrames_in_PySpark_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search and Filter DataFrames in PySpark HW

Now it's time to put what you've learn into action with a homework assignment!

In case you need it again, here is the link to the documentation for the full list available function in pyspark.sql.functions library:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions


### First set up your Spark Session!
Alright so first things first, let's start up our pyspark instance.

In [ ]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Select").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the DataFrame for this Notebook

We will be continuing to use the fifa19.csv file for this notebook. Make sure that you are writting the correct path to the file.

In [ ]:
path = 'Datasets/'
fifa = spark.read.csv(path+'fifa19.csv',inferSchema=True,header=True)

## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below:

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB,
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

Use the .toPandas() method to view the first few lines of the dataset so we know what we are working with.

In [ ]:
fifa.limit(4).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68,15,21,13,90,85,87,88,94,€138.6M


Now print the schema of the dataset so we can see the data types of all the varaibles.

In [ ]:
print(fifa.printSchema())

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

## Now let's get started!

### First things first..... import the pyspark sql functions library

Since we know we will be using it a lot.

In [ ]:
from pyspark.sql.functions import *

### 1. Select the Name and Position of each player in the dataframe

In [ ]:
fifa.select(['Name','Position']).show(5)

+-----------------+--------+
|             Name|Position|
+-----------------+--------+
|         L. Messi|      RF|
|Cristiano Ronaldo|      ST|
|        Neymar Jr|      LW|
|           De Gea|      GK|
|     K. De Bruyne|     RCM|
+-----------------+--------+
only showing top 5 rows



### 1.1 Display the same results from above sorted by the players names

In [ ]:
fifa.select(['Name']).show(5)

+-----------------+
|             Name|
+-----------------+
|         L. Messi|
|Cristiano Ronaldo|
|        Neymar Jr|
|           De Gea|
|     K. De Bruyne|
+-----------------+
only showing top 5 rows



### 2. Select only the players who belong to a club begining with FC

In [ ]:
fifa.select("Name","Club").where(fifa.Club.like("%FC%")).show(5, False)

+---------------+-----------------+
|Name           |Club             |
+---------------+-----------------+
|L. Messi       |FC Barcelona     |
|L. Suárez      |FC Barcelona     |
|R. Lewandowski |FC Bayern München|
|M. ter Stegen  |FC Barcelona     |
|Sergio Busquets|FC Barcelona     |
+---------------+-----------------+
only showing top 5 rows



In [ ]:
fifa.select("Name","Club").where(fifa.Club.like("%FC%")).show()

+---------------+--------------------+
|           Name|                Club|
+---------------+--------------------+
|       L. Messi|        FC Barcelona|
|      L. Suárez|        FC Barcelona|
| R. Lewandowski|   FC Bayern München|
|  M. ter Stegen|        FC Barcelona|
|Sergio Busquets|        FC Barcelona|
|       M. Neuer|   FC Bayern München|
|   J. Rodríguez|   FC Bayern München|
|       Coutinho|        FC Barcelona|
|     M. Hummels|   FC Bayern München|
|      S. Umtiti|        FC Barcelona|
|     Jordi Alba|        FC Barcelona|
|     I. Rakitić|        FC Barcelona|
|          Piqué|        FC Barcelona|
|      T. Müller|   FC Bayern München|
|         Thiago|   FC Bayern München|
|     J. Kimmich|   FC Bayern München|
|       D. Alaba|   FC Bayern München|
|       Paulinho|Guangzhou Evergra...|
|     Y. Brahimi|            FC Porto|
|     J. Boateng|   FC Bayern München|
+---------------+--------------------+
only showing top 20 rows



### 3. Who is the oldest player in the dataset and how old are they?

Display only the name and age of the oldest player.

In [ ]:
fifa.select(['Nationality','Name','Age']).orderBy(fifa["Age"].desc()).show(5)

+-----------------+-------------+---+
|      Nationality|         Name|Age|
+-----------------+-------------+---+
|           Mexico|     O. Pérez| 45|
|          England|K. Pilkington| 44|
|Trinidad & Tobago|    T. Warner| 44|
|            Japan|  S. Narazaki| 42|
|         Paraguay|    J. Villar| 41|
+-----------------+-------------+---+
only showing top 5 rows



### 4. Select only the following players from the dataframe:

 - L. Messi
 - Cristiano Ronaldo

In [ ]:
fifa[fifa.Name.isin("L. Messi","Cristiano Ronaldo")].limit(4).toPandas()

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M


### 5. Can you select the first character from the Release Clause variable which indicates the currency used?

In [ ]:
renamed = fifa.withColumnRenamed('Release Clause','ReleaseClause')

In [ ]:
renamed.select("ReleaseClause",renamed.ReleaseClause.substr(1,1)).show(5,False)

+-------------+------------------------------+
|ReleaseClause|substring(ReleaseClause, 1, 1)|
+-------------+------------------------------+
|€226.5M      |€                             |
|€127.1M      |€                             |
|€228.1M      |€                             |
|€138.6M      |€                             |
|€196.4M      |€                             |
+-------------+------------------------------+
only showing top 5 rows



### 6. Can you select only the players who are over the age of 40?

In [ ]:
fifa.filter("Age>40").select(['ID','Name','Nationality','Age']).limit(10).toPandas()

,ID,Name,Nationality,Age
0,156092,J. Villar,Paraguay,41
1,3665,B. Nivet,France,41
2,140029,O. Pérez,Mexico,45
3,142998,C. Muñoz,Argentina,41
4,140183,S. Narazaki,Japan,42
5,208927,H. Sulaimani,Saudi Arabia,41
6,18745,M. Tyler,England,41
7,51963,T. Warner,Trinidad & Tobago,44
8,53748,K. Pilkington,England,44


### That's is for now... Great Job!